In [4]:
import os
import h5py
import tifffile
import numpy as np
import webknossos as wk
from webknossos.geometry.mag import Mag

from matplotlib import pyplot as plt

### Convert BDV/HDF5 file to webKnossos
---
Inputs:

In [10]:
h5_file = "B:/fs3-bifexchgrp/BIF_StaffSci/Christoph/danzlgrp/Jules/PrePost2/fused/post_n2v_fused.h5"
BLOCK_SIZE=1024

### Helper functions

In [11]:
def get_h5ds(h5_file, c, mag):
    hf = h5py.File(h5_file, "r")

    df = hf[f"t00000/s0{c}/{mag}/cells"]
    z, y, x = df.shape

    return df

def my_block_iter(shape, block_size):
    z_slc = slice(0, shape[0])
    blocks_y = shape[1] // block_size
    blocks_x = shape[2] // block_size
    
    for by in range(blocks_y+1):
        y_slc = slice(by*block_size,min(shape[1],(by+1)*block_size))
        for bx in range(blocks_x+1):
            x_slc = slice(bx*block_size,min(shape[2],(bx+1)*block_size))
            yield z_slc, y_slc, x_slc

### Output and intensity rescaling

In [12]:
dataset = wk.Dataset("B:/fs3-bifexchgrp/BIF_StaffSci/Christoph/danzlgrp/Jules/PrePost2/fused/post_wk_uint8/", scale=(150, 150, 150))

In [13]:
min_maxs = [(100, 355), (64500, 65536), (100,355)]

### Convert

In [ ]:
for c in range(3):
    h5ds = get_h5ds(h5_file, c=c, mag=0)
    print(h5ds.shape)

    layer = dataset.add_layer(
        layer_name=f"channel_{c}", category="color", dtype_per_channel="uint8", num_channels=1
    )

    mag1 = layer.add_mag("1")
    print("Channel", c)
    
    mi, ma = min_maxs[c]

    for z_slc, y_slc, x_slc in my_block_iter(h5ds.shape, BLOCK_SIZE):
        z_start = z_slc.start
        y_start = y_slc.start
        x_start = x_slc.start

        data = h5ds[z_slc, y_slc, x_slc].astype("uint16")
        data = data.swapaxes(0, 2)
        
        data = (255*((data.astype("float32") - mi) / (ma - mi))).astype("uint8")

        mag1.write(data=data, absolute_offset=(x_start, y_start, z_start))
        print(".", end="")

    layer.downsample(
        Mag(1),
        Mag(4),
    )    
    
        